In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import os,sys,inspect
import gc
from tqdm import tqdm
import random

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

from load import *
from evals import *

import warnings
warnings.filterwarnings('ignore')

In [2]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import optimizers, callbacks, layers, losses
from tensorflow.keras.layers import Dense, Concatenate, Activation, Add, BatchNormalization, Dropout, Input, Embedding, Flatten, Multiply
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)
        
def mish(x):
    return x*tf.math.tanh(tf.math.softplus(x))

def leakyrelu(x, factor=0.2):
    return tf.maximum(x, factor*x)

## Load

In [3]:
df = load_data('../data/ml-100k/u.data', threshold=3)
uuid = df['userId'].unique()
uiid = df['movieId'].unique()


In [4]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.15, random_state=SEED, stratify=df['userId'].values)

In [5]:
tr_X = np.stack([train['userId'].values.astype(np.int32), train['movieId'].values.astype(np.int32)], 1)
test_X = np.stack([test['userId'].values.astype(np.int32), test['movieId'].values.astype(np.int32)], 1)

tr_X.shape, test_X.shape

((85000, 2), (15000, 2))

## Model

In [6]:
class FM_layer(keras.Model):
    def __init__(self, latent_dim, w_reg=1e-4, v_reg=1e-4):
        super(FM_layer, self).__init__()
        self.latent_dim = latent_dim
        
        self.w_reg = w_reg
        self.v_reg = v_reg

    def build(self, input_shape):
        self.w_0 = self.add_weight(shape=(1, ),
                                  initializer=tf.zeros_initializer(),
                                  trainable=True)
        
        self.w = self.add_weight(shape=(input_shape[-1], 1), 
                             initializer=tf.random_normal_initializer(),
                                trainable=True,
                                regularizer=l2(self.w_reg))
        
        self.V = self.add_weight(shape=(input_shape[-1], self.latent_dim), 
                             initializer=tf.random_normal_initializer(),
                                trainable=True,
                                regularizer=l2(self.v_reg))

    def call(self, inputs):
        linear_terms = tf.reduce_sum(tf.matmul(inputs, self.w), axis=1)

        interactions = 0.5 * tf.reduce_sum(
            tf.pow(tf.matmul(inputs, self.V), 2)
            - tf.matmul(tf.pow(inputs, 2), tf.pow(self.V, 2)),
            1,
            keepdims=False
        )

        y_hat = (self.w_0 + linear_terms + interactions)

        return y_hat

In [7]:
class DeepFM(tf.keras.Model):
    def __init__(self, x_dims, latent_dim, l2_emb=1e-4):
        super(DeepFM, self).__init__()
        
        self.x_dims = x_dims
        self.latent_dim = latent_dim

        self.embedding = Embedding(sum(x_dims)+1, latent_dim, input_length=1, embeddings_regularizer=l2(l2_emb))
        self.fm_layer = FM_layer(latent_dim)
        self.dnn_layers = self.build_dnn()
        self.flatten =  Flatten()

    def build_dnn(self):
        model = Sequential()
        model.add(Dense(128, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(1))
        
        return model
        

    def call(self, inputs):        
        emb = self.embedding(inputs + tf.constant((0, *np.cumsum(self.x_dims)))[:-1])
        embed = self.flatten(emb)

        fm_out = self.fm_layer(embed)

        deep_out = self.dnn_layers(embed)

        outputs = fm_out + tf.squeeze(deep_out)
        
        return outputs

## Train

In [8]:
dfm = DeepFM((len(uuid), len(uiid)), 8)

In [9]:
dfm.compile(loss=losses.BinaryCrossentropy(from_logits=True), 
            optimizer=optimizers.Adam())

dfm.fit(tr_X, 
       train['rating'].values,
      epochs=10,
      shuffle=True,
      validation_split=0.1)

Epoch 1/10
2391/2391 [==============================] - 11s 5ms/step - loss: 0.5958 - val_loss: 0.5773
Epoch 2/10
2391/2391 [==============================] - 10s 4ms/step - loss: 0.5615 - val_loss: 0.5757
Epoch 3/10
2391/2391 [==============================] - 10s 4ms/step - loss: 0.5545 - val_loss: 0.5728
Epoch 4/10
2391/2391 [==============================] - 10s 4ms/step - loss: 0.5495 - val_loss: 0.5696
Epoch 5/10
2391/2391 [==============================] - 10s 4ms/step - loss: 0.5443 - val_loss: 0.5689
Epoch 6/10
2391/2391 [==============================] - 10s 4ms/step - loss: 0.5365 - val_loss: 0.5712
Epoch 7/10
2391/2391 [==============================] - 10s 4ms/step - loss: 0.5299 - val_loss: 0.5777
Epoch 8/10
2391/2391 [==============================] - 10s 4ms/step - loss: 0.5234 - val_loss: 0.5752
Epoch 9/10
2391/2391 [==============================] - 10s 4ms/step - loss: 0.5178 - val_loss: 0.5755
Epoch 10/10
2391/2391 [==============================] - 10s 4ms/step - l

## Eval

In [10]:
pred = dfm.predict(test_X)


In [11]:
np.sum(np.where(pred>0., 1, 0).flatten() == test['rating'].values) / len(pred)

0.7224

In [12]:
from sklearn.metrics import precision_score, recall_score,  roc_auc_score, precision_recall_fscore_support

print(roc_auc_score(test['rating'].values, pred))
print(precision_score(test['rating'].values, np.where(pred>0., 1, 0)))
print(recall_score(test['rating'].values, np.where(pred>0., 1, 0)))

0.7888386196000222
0.7348484848484849
0.7813176244289493
